In [12]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec  4 15:11:48 2018

@author: Fang
"""

import pandas as pd
import numpy as np
from sqlalchemy import *
from sqlalchemy import create_engine
import os
import datetime as dt
import time
import urllib.request as urlreq
import json

module_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\Modules\\DataCollection'
os.chdir(module_dir)


def hist_prices(ticker, start_date, end_date = dt.datetime.today()):
    start_date_unix = int(start_date.timestamp())
    end_date = dt.datetime.today()
    end_date_unix = int(end_date.timestamp())

    hist_price_url = 'https://query1.finance.yahoo.com/v8/finance/chart/{0}?symbol={0}&period1={1}&period2={2}&interval=1d'.format(ticker,start_date_unix,end_date_unix)
    #print(hist_price_url)
    with urlreq.urlopen(hist_price_url) as url:
        data = json.loads(url.read().decode())
        hist_prices = pd.DataFrame({**data['chart']['result'][0]['indicators']['quote'][0], **data['chart']['result'][0]['indicators']['adjclose'][0]},
                                        index = [dt.datetime.utcfromtimestamp(int(x)).date() for x in 
                                                 data['chart']['result'][0]['timestamp']])
        hist_prices.index = pd.to_datetime(hist_prices.index)
    
    return hist_prices

start_date = dt.datetime(1999,1,1)

# Initializing Stock Universe
os.chdir('..\\')
os.chdir('..\\')
os.chdir('..\\Data\\Stock Universe')
#tsx = pd.read_csv('TSX.csv')['Symbol']#.tolist()
nyse = pd.read_csv('NYSE.csv')['Symbol']#.tolist()
nasdaq = pd.read_csv('NASDAQ.csv')['Symbol']#.tolist()
amex = pd.read_csv('AMEX.csv')['Symbol']#.tolist()
us_names = pd.concat([nyse,nasdaq,amex]).drop_duplicates().values.tolist()

del nyse, nasdaq, amex

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir(dbs_dir)

earn_engine = create_engine('sqlite:///earningsHistory.db', echo=False)
yahoo_engine = create_engine('sqlite:///yahoo.db', echo = False)

prices_db_dir = 'D:\\Price Data'
os.chdir(prices_db_dir)
hisprices_engine = create_engine('sqlite:///histprices.db', echo = False)

run_update = True

def return_profiles(ticker):
    start_time = time.time()
    prices_query = 'SELECT Date, adjclose FROM historicalPrices WHERE Underlying = "{}"'.format(ticker)
    curr_closes = pd.read_sql_query(prices_query, con=hisprices_engine, index_col = 'Date')
    
    if len(curr_closes) == 0:
        return (ticker,)*8
    curr_closes.index = pd.to_datetime(curr_closes.index)

    yahoo_db_query = 'SELECT Underlying, industry, sector FROM profiles WHERE Underlying = "{}"'.format(ticker)
    curr_yahoo_db = pd.read_sql_query(yahoo_db_query, con = yahoo_engine, index_col = 'Underlying').reset_index().drop_duplicates('Underlying').reset_index(drop = True)
    if len(curr_yahoo_db) == 0:
        curr_yahoo_db = pd.DataFrame({'Underlying': [ticker], 'industry': [np.nan], 'sector': [np.nan]})
        
    quarter_mapping = {1: 1, 2:1, 3:1, 
                       4:2, 5:2, 6:2,
                       7:3, 8:3, 9:3,
                       10:4, 11:4, 12:4}

    curr_closes['year'] = curr_closes.index.year
    curr_closes['month'] = curr_closes.index.month
    curr_closes['boy'] = curr_closes['year'] - curr_closes['year'].shift(1)
    curr_closes['bom'] = curr_closes['month'] - curr_closes['month'].shift(1)
    curr_closes['boq'] = curr_closes['month'] % 3
    curr_closes['quarter'] = curr_closes['month'].replace(quarter_mapping)
    curr_closes['Q'] = curr_closes['year'].apply(lambda x: str(x)) + ' Q' + curr_closes['quarter'].apply(lambda x: str(x))
    curr_closes['dailyReturns'] = curr_closes['adjclose'].pct_change()

    annual_returns = curr_closes[curr_closes['boy'] != 0][['adjclose','year']]
    annual_returns['adjclose'] = annual_returns['adjclose'].pct_change().shift(-1)
    annual_returns = annual_returns.set_index('year').dropna()
    annual_returns.columns = ['return']

    quarterly_returns = curr_closes[curr_closes['bom'] != 0][['adjclose','boq','Q']]
    quarterly_returns = quarterly_returns[quarterly_returns['boq'] == 1][['adjclose','Q']]
    quarterly_returns['adjclose'] = quarterly_returns['adjclose'].pct_change().shift(-1)
    quarterly_returns = quarterly_returns.set_index('Q').dropna()
    quarterly_returns.columns = ['return']

    def risk_profiles(returns_df, curr_closes, curr_yahoo_db):
        maxDrawDowns = pd.DataFrame(index = returns_df.index, columns = ['maxDrawDowns'])
        downsideVols = pd.DataFrame(index = returns_df.index, columns = ['downsideVols'])
        returnVols = pd.DataFrame(index = returns_df.index, columns = ['vol'])

        for period in returns_df.index:
            curr_year = curr_closes[curr_closes[returns_df.index.name] == period][['adjclose','dailyReturns']]
            curr_year['cumReturns'] = (curr_year['dailyReturns'] + 1).cumprod()
            curr_year['rollingMax'] = curr_year['cumReturns'].expanding(min_periods=1).max()
            curr_year['drawDown'] = curr_year['cumReturns']/curr_year['rollingMax'] - 1
            curr_downside = curr_year[curr_year['dailyReturns'] < 0]

            maxDrawDowns.loc[period,'maxDrawDowns'] = curr_year['drawDown'].min()
            downsideVols.loc[period,'downsideVols'] = curr_downside['dailyReturns'].std()*np.sqrt(len(curr_downside))
            returnVols.loc[period, 'vol'] = curr_year['dailyReturns'].std()*np.sqrt(252)

        maxDrawDowns, downsideVols, returnVols = maxDrawDowns.T, downsideVols.T, returnVols.T
        maxDrawDowns.index, downsideVols.index, returnVols.index = curr_yahoo_db.Underlying, curr_yahoo_db.Underlying, curr_yahoo_db.Underlying
        maxDrawDowns[['industry','sector']], downsideVols[['industry','sector']], returnVols[['industry','sector']] = curr_yahoo_db[['industry','sector']], curr_yahoo_db[['industry','sector']], curr_yahoo_db[['industry','sector']]
        maxDrawDowns[['industry','sector']], downsideVols[['industry','sector']], returnVols[['industry','sector']] = curr_yahoo_db.iloc[:,1:].values, curr_yahoo_db.iloc[:,1:].values, curr_yahoo_db.iloc[:,1:].values
        return  maxDrawDowns, downsideVols, returnVols

    annualDDs, annualDownsides, annualVols = risk_profiles(annual_returns, curr_closes, curr_yahoo_db)
    quarterlyDDs, quarterlyDownsides, quarterlyVols = risk_profiles(quarterly_returns, curr_closes, curr_yahoo_db)
    annual_returns, quarterly_returns = annual_returns.T, quarterly_returns.T
    annual_returns.index, quarterly_returns.index = curr_yahoo_db.Underlying, curr_yahoo_db.Underlying
    annual_returns[['industry','sector']], quarterly_returns[['industry','sector']] = curr_yahoo_db[['industry','sector']], curr_yahoo_db[['industry','sector']]
    annual_returns[['industry','sector']], quarterly_returns[['industry','sector']] = curr_yahoo_db.iloc[:,1:].values, curr_yahoo_db.iloc[:,1:].values

    print("Completed in %s seconds" % (time.time() - start_time))
    
    return annual_returns, annualDDs, annualDownsides, annualVols, quarterly_returns, quarterlyDDs, quarterlyDownsides, quarterlyVols



if run_update:
    
    start_time = time.time()
    i = 0
    total_length = len(us_names)
    for ticker in us_names:
        

        try:
            curr_hist_prices = hist_prices(ticker, start_date)
            curr_hist_prices = curr_hist_prices.sort_index(ascending = True)
            curr_hist_prices['Underlying'] = ticker
        except:
            print('Failed prices for {}'.format(ticker))
            print('{0:.2f}% Completed'.format(i/total_length*100))
            continue

        table_in_db = True

        try:
            query = 'SELECT Date, Underlying FROM historicalPrices WHERE Underlying = "{}"'.format(ticker)
            curr_hist_db = pd.read_sql_query(query, con=hisprices_engine, index_col = 'Date').tail(1)
            curr_hist_db.index = pd.to_datetime(curr_hist_db.index)
        except:
            table_in_db = False

        if table_in_db and len(curr_hist_db) != 0:
            latest_date_in_db = curr_hist_db.index.tolist()[0]
            curr_hist_prices = curr_hist_prices[curr_hist_prices.index > latest_date_in_db]

        curr_hist_prices.to_sql('historicalPrices', con=hisprices_engine, 
                                if_exists='append', index_label = 'Date')
        i += 1
        print('{0:.2f}% Completed'.format(i/total_length*100))

    print("Completed in %s seconds" % (time.time() - start_time))

0.01% Completed
0.02% Completed
0.03% Completed
0.05% Completed
0.06% Completed
0.07% Completed
0.08% Completed
0.09% Completed
0.10% Completed
0.12% Completed
0.13% Completed
0.14% Completed
0.15% Completed
0.16% Completed
0.17% Completed
Failed prices for ABR-A
0.17% Completed
Failed prices for ABR-B
0.17% Completed
Failed prices for ABR-C
0.17% Completed
0.18% Completed
0.20% Completed
0.21% Completed
0.22% Completed
0.23% Completed
0.24% Completed
0.25% Completed
0.27% Completed
0.28% Completed
0.29% Completed
0.30% Completed
0.31% Completed
0.32% Completed
0.33% Completed
0.35% Completed
0.36% Completed
0.37% Completed
0.38% Completed
0.39% Completed
0.40% Completed
0.42% Completed
0.43% Completed
0.44% Completed
0.45% Completed
0.46% Completed
0.47% Completed
0.48% Completed
0.50% Completed
0.51% Completed
0.52% Completed
0.53% Completed
0.54% Completed
0.55% Completed
0.57% Completed
0.58% Completed
0.59% Completed
0.60% Completed
Failed prices for AFS-A
0.60% Completed
Failed p

3.57% Completed
3.58% Completed
3.59% Completed
3.60% Completed
3.61% Completed
3.62% Completed
3.64% Completed
3.65% Completed
3.66% Completed
3.67% Completed
3.68% Completed
3.69% Completed
3.71% Completed
3.72% Completed
3.73% Completed
3.74% Completed
3.75% Completed
3.76% Completed
3.77% Completed
3.79% Completed
Failed prices for BML-G
3.79% Completed
Failed prices for BML-H
3.79% Completed
Failed prices for BML-J
3.79% Completed
Failed prices for BML-L
3.79% Completed
3.80% Completed
3.81% Completed
3.82% Completed
3.83% Completed
3.84% Completed
3.86% Completed
3.87% Completed
3.88% Completed
3.89% Completed
3.90% Completed
3.91% Completed
3.92% Completed
3.94% Completed
3.95% Completed
3.96% Completed
3.97% Completed
3.98% Completed
3.99% Completed
4.01% Completed
4.02% Completed
4.03% Completed
Failed prices for BRK.A
4.03% Completed
Failed prices for BRK.B
4.03% Completed
4.04% Completed
4.05% Completed
4.06% Completed
4.07% Completed
4.09% Completed
4.10% Completed
4.11% Co

Failed prices for DDR-J
7.47% Completed
Failed prices for DDR-K
7.47% Completed
7.48% Completed
7.49% Completed
7.50% Completed
7.51% Completed
7.53% Completed
7.54% Completed
7.55% Completed
7.56% Completed
7.57% Completed
7.58% Completed
7.60% Completed
7.61% Completed
7.62% Completed
7.63% Completed
7.64% Completed
Failed prices for DHC.X
7.64% Completed
7.65% Completed
7.66% Completed
7.68% Completed
7.69% Completed
7.70% Completed
7.71% Completed
7.72% Completed
7.73% Completed
7.75% Completed
7.76% Completed
7.77% Completed
7.78% Completed
7.79% Completed
7.80% Completed
7.81% Completed
Failed prices for DLN-A
7.81% Completed
7.83% Completed
7.84% Completed
7.85% Completed
Failed prices for DLR-C
7.85% Completed
Failed prices for DLR-G
7.85% Completed
Failed prices for DLR-H
7.85% Completed
Failed prices for DLR-I
7.85% Completed
Failed prices for DLR-J
7.85% Completed
7.86% Completed
7.87% Completed
7.88% Completed
7.90% Completed
7.91% Completed
7.92% Completed
7.93% Completed


Failed prices for GGT-E
11.70% Completed
11.72% Completed
Failed prices for GGZ-A
11.72% Completed
11.73% Completed
11.74% Completed
11.75% Completed
11.76% Completed
11.77% Completed
11.79% Completed
11.80% Completed
Failed prices for GIG.P
11.80% Completed
Failed prices for GIG.U
11.80% Completed
Failed prices for GIG.W
11.80% Completed
11.81% Completed
11.82% Completed
11.83% Completed
11.84% Completed
11.86% Completed
11.87% Completed
11.88% Completed
11.89% Completed
11.90% Completed
11.91% Completed
11.92% Completed
Failed prices for GLO-A
11.92% Completed
Failed prices for GLO-B
11.92% Completed
Failed prices for GLO-G
11.92% Completed
11.94% Completed
11.95% Completed
11.96% Completed
11.97% Completed
Failed prices for GLP-A
11.97% Completed
11.98% Completed
11.99% Completed
12.01% Completed
Failed prices for GM.B
12.01% Completed
12.02% Completed
12.03% Completed
Failed prices for GMR-A
12.03% Completed
12.04% Completed
12.05% Completed
12.06% Completed
12.07% Completed
12.09%

15.49% Completed
15.50% Completed
15.51% Completed
15.53% Completed
15.54% Completed
15.55% Completed
15.56% Completed
15.57% Completed
Failed prices for KEY-I
15.57% Completed
Failed prices for KEY-J
15.57% Completed
15.58% Completed
15.60% Completed
15.61% Completed
15.62% Completed
15.63% Completed
15.64% Completed
Failed prices for KIM-I
15.64% Completed
Failed prices for KIM-J
15.64% Completed
Failed prices for KIM-K
15.64% Completed
Failed prices for KIM-L
15.64% Completed
Failed prices for KIM-M
15.64% Completed
15.65% Completed
15.66% Completed
Failed prices for KKR-A
15.66% Completed
Failed prices for KKR-B
15.66% Completed
15.68% Completed
15.69% Completed
15.70% Completed
15.71% Completed
15.72% Completed
Failed prices for KMI-A
15.72% Completed
15.73% Completed
15.75% Completed
15.76% Completed
15.77% Completed
15.78% Completed
15.79% Completed
15.80% Completed
15.81% Completed
15.83% Completed
15.84% Completed
Failed prices for KOD.W
15.84% Completed
Failed prices for KOD.

...

68.53% Completed
68.54% Completed
68.56% Completed
68.57% Completed
68.58% Completed
68.59% Completed
68.60% Completed
68.61% Completed
68.63% Completed
68.64% Completed
68.65% Completed
68.66% Completed
68.67% Completed
68.68% Completed
68.69% Completed
68.71% Completed
68.72% Completed
68.73% Completed
68.74% Completed
68.75% Completed
68.76% Completed
68.78% Completed
68.79% Completed
68.80% Completed
68.81% Completed
68.82% Completed
68.83% Completed
68.84% Completed
68.86% Completed
68.87% Completed
68.88% Completed
68.89% Completed
68.90% Completed
68.91% Completed
68.93% Completed
68.94% Completed
68.95% Completed
68.96% Completed
68.97% Completed
68.98% Completed
68.99% Completed
69.01% Completed
69.02% Completed
69.03% Completed
69.04% Completed
69.05% Completed
69.06% Completed
69.08% Completed
69.09% Completed
69.10% Completed
69.11% Completed
69.12% Completed
69.13% Completed
69.14% Completed
69.16% Completed
69.17% Completed
69.18% Completed
69.19% Completed
69.20% Complet

73.80% Completed
73.81% Completed
73.82% Completed
73.83% Completed
73.84% Completed
73.85% Completed
73.87% Completed
73.88% Completed
73.89% Completed
73.90% Completed
73.91% Completed
73.92% Completed
73.94% Completed
73.95% Completed
73.96% Completed
73.97% Completed
73.98% Completed
73.99% Completed
74.00% Completed
74.02% Completed
74.03% Completed
74.04% Completed
74.05% Completed
74.06% Completed
74.07% Completed
74.09% Completed
74.10% Completed
74.11% Completed
74.12% Completed
74.13% Completed
74.14% Completed
74.15% Completed
74.17% Completed
74.18% Completed
74.19% Completed
74.20% Completed
74.21% Completed
74.22% Completed
74.24% Completed
74.25% Completed
74.26% Completed
74.27% Completed
74.28% Completed
74.29% Completed
74.30% Completed
74.32% Completed
74.33% Completed
74.34% Completed
74.35% Completed
74.36% Completed
74.37% Completed
74.39% Completed
74.40% Completed
74.41% Completed
74.42% Completed
74.43% Completed
74.44% Completed
74.45% Completed
74.47% Complet

79.08% Completed
79.10% Completed
79.11% Completed
79.12% Completed
79.13% Completed
79.14% Completed
79.15% Completed
79.16% Completed
79.18% Completed
79.19% Completed
79.20% Completed
79.21% Completed
79.22% Completed
79.23% Completed
79.25% Completed
79.26% Completed
79.27% Completed
79.28% Completed
79.29% Completed
79.30% Completed
79.31% Completed
79.33% Completed
79.34% Completed
79.35% Completed
79.36% Completed
79.37% Completed
79.38% Completed
79.40% Completed
79.41% Completed
79.42% Completed
79.43% Completed
79.44% Completed
79.45% Completed
79.46% Completed
79.48% Completed
79.49% Completed
79.50% Completed
79.51% Completed
79.52% Completed
79.53% Completed
79.55% Completed
79.56% Completed
79.57% Completed
79.58% Completed
79.59% Completed
79.60% Completed
79.61% Completed
79.63% Completed
79.64% Completed
79.65% Completed
79.66% Completed
79.67% Completed
79.68% Completed
79.70% Completed
79.71% Completed
79.72% Completed
79.73% Completed
79.74% Completed
79.75% Complet

84.46% Completed
84.47% Completed
84.49% Completed
84.50% Completed
84.51% Completed
84.52% Completed
84.53% Completed
84.54% Completed
84.56% Completed
84.57% Completed
84.58% Completed
84.59% Completed
84.60% Completed
84.61% Completed
84.62% Completed
84.64% Completed
84.65% Completed
84.66% Completed
84.67% Completed
Failed prices for PCG-A
84.67% Completed
Failed prices for PCG-B
84.67% Completed
Failed prices for PCG-C
84.67% Completed
Failed prices for PCG-D
84.67% Completed
Failed prices for PCG-E
84.67% Completed
Failed prices for PCG-G
84.67% Completed
Failed prices for PCG-H
84.67% Completed
Failed prices for PCG-I
84.67% Completed
84.68% Completed
84.69% Completed
84.71% Completed
84.72% Completed
84.73% Completed
84.74% Completed
84.75% Completed
84.76% Completed
84.77% Completed
84.79% Completed
84.80% Completed
84.81% Completed
84.82% Completed
84.83% Completed
84.84% Completed
84.86% Completed
84.87% Completed
84.88% Completed
84.89% Completed
84.90% Completed
84.91% Co

89.53% Completed
89.54% Completed
89.55% Completed
89.56% Completed
89.58% Completed
89.59% Completed
89.60% Completed
89.61% Completed
89.62% Completed
89.63% Completed
89.65% Completed
89.66% Completed
89.67% Completed
89.68% Completed
89.69% Completed
89.70% Completed
89.71% Completed
89.73% Completed
89.74% Completed
89.75% Completed
89.76% Completed
89.77% Completed
89.78% Completed
89.80% Completed
89.81% Completed
89.82% Completed
89.83% Completed
89.84% Completed
89.85% Completed
89.86% Completed
89.88% Completed
89.89% Completed
89.90% Completed
89.91% Completed
89.92% Completed
89.93% Completed
89.95% Completed
89.96% Completed
89.97% Completed
89.98% Completed
89.99% Completed
90.00% Completed
90.02% Completed
90.03% Completed
90.04% Completed
90.05% Completed
90.06% Completed
90.07% Completed
90.08% Completed
90.10% Completed
90.11% Completed
90.12% Completed
90.13% Completed
90.14% Completed
90.15% Completed
90.17% Completed
90.18% Completed
90.19% Completed
90.20% Complet

In [9]:
# spx = hist_prices('^GSPC', start_date)
# spx = spx.sort_index(ascending = True)
# spx['Underlying'] = 'SPX'
# spx = spx[['Underlying','adjclose']]
# spx['industry'] = 'SP500'
# spx['sector'] = 'Index'

In [14]:
start_time = time.time()
i = 0
total_length = len(us_names)
return_engine = create_engine('sqlite:///returnProfiles.db', echo = False)

for ticker in us_names:
    annual_returns, annualDDs, annualDownsides, annualVols, quarterly_returns, quarterlyDDs, quarterlyDownsides, quarterlyVols = return_profiles(ticker)
    if type(annual_returns) != str and ticker != 'AAPL':
        annual_returns.to_sql('annualReturns', con=return_engine, 
                              if_exists='append', index_label = 'Underlying')
        annualDDs.to_sql('annualDDs', con=return_engine, 
                         if_exists='append', index_label = 'Underlying')
        annualDownsides.to_sql('annualDownsides', con=return_engine, 
                               if_exists='append', index_label = 'Underlying')
        annualVols.to_sql('annualVols', con=return_engine, 
                          if_exists='append', index_label = 'Underlying')
        
        quarterly_returns.to_sql('quarterlyReturns', con=return_engine, 
                                 if_exists='append', index_label = 'Underlying')
        quarterlyDDs.to_sql('quarterlyDDs', con=return_engine, 
                            if_exists='append', index_label = 'Underlying')
        quarterlyDownsides.to_sql('quarterlyDownsides', con=return_engine, 
                                  if_exists='append', index_label = 'Underlying')
        quarterlyVols.to_sql('quarterlyVols', con=return_engine, 
                             if_exists='append', index_label = 'Underlying')    
    i += 1
    print('{0:.2f}% Completed'.format(i/total_length*100))

print("Completed in %s seconds" % (time.time() - start_time))

Completed in 6.330118179321289 seconds
0.01% Completed
Completed in 6.595409393310547 seconds
0.02% Completed
Completed in 3.9264814853668213 seconds
0.03% Completed
Completed in 6.447808504104614 seconds
0.05% Completed
Completed in 6.028916120529175 seconds
0.06% Completed
Completed in 4.770247220993042 seconds
0.07% Completed
Completed in 3.663210868835449 seconds
0.08% Completed
Completed in 6.467751502990723 seconds
0.09% Completed
Completed in 6.570444583892822 seconds
0.10% Completed
Completed in 4.724377393722534 seconds
0.12% Completed
Completed in 6.439794063568115 seconds
0.13% Completed
Completed in 6.304155588150024 seconds
0.14% Completed
Completed in 5.94410252571106 seconds
0.15% Completed
Completed in 6.3610334396362305 seconds
0.16% Completed
Completed in 5.790525674819946 seconds
0.17% Completed
0.18% Completed
0.20% Completed
0.21% Completed
Completed in 6.791853427886963 seconds
0.22% Completed
Completed in 6.774911165237427 seconds
0.23% Completed
Completed in 5.4

2.09% Completed
Completed in 6.971394300460815 seconds
2.10% Completed
Completed in 4.297518491744995 seconds
2.11% Completed
Completed in 4.914867877960205 seconds
2.12% Completed
2.14% Completed
2.15% Completed
Completed in 3.929502248764038 seconds
2.16% Completed
Completed in 4.707422256469727 seconds
2.17% Completed
Completed in 4.651605844497681 seconds
2.18% Completed
Completed in 6.905513525009155 seconds
2.19% Completed
Completed in 3.9534378051757812 seconds
2.20% Completed
Completed in 3.8287734985351562 seconds
2.22% Completed
Completed in 5.129295349121094 seconds
2.23% Completed
Completed in 3.982337474822998 seconds
2.24% Completed
Completed in 4.6515727043151855 seconds
2.25% Completed
Completed in 6.485665798187256 seconds
2.26% Completed
Completed in 5.630954265594482 seconds
2.27% Completed
Completed in 4.156895399093628 seconds
2.29% Completed
Completed in 4.9507386684417725 seconds
2.30% Completed
Completed in 4.07311749458313 seconds
2.31% Completed
Completed in 4

Completed in 6.28623628616333 seconds
4.10% Completed
Completed in 6.275231838226318 seconds
4.11% Completed
Completed in 4.682470321655273 seconds
4.12% Completed
Completed in 3.6871488094329834 seconds
4.13% Completed
Completed in 6.463759660720825 seconds
4.14% Completed
Completed in 4.670552968978882 seconds
4.16% Completed
Completed in 3.826749801635742 seconds
4.17% Completed
Completed in 5.709738492965698 seconds
4.18% Completed
Completed in 5.467391490936279 seconds
4.19% Completed
Completed in 5.959104299545288 seconds
4.20% Completed
Completed in 3.790841817855835 seconds
4.21% Completed
Completed in 5.617994070053101 seconds
4.22% Completed
Completed in 6.609308958053589 seconds
4.24% Completed
4.25% Completed
Completed in 3.615344524383545 seconds
4.26% Completed
Completed in 6.245313405990601 seconds
4.27% Completed
Completed in 6.274235963821411 seconds
4.28% Completed
Completed in 6.158546209335327 seconds
4.29% Completed
Completed in 6.144582033157349 seconds
4.31% Comp

Completed in 6.2523040771484375 seconds
5.99% Completed
Completed in 6.674168109893799 seconds
6.00% Completed
6.01% Completed
6.03% Completed
Completed in 6.405885457992554 seconds
6.04% Completed
Completed in 3.7360172271728516 seconds
6.05% Completed
Completed in 4.1309897899627686 seconds
6.06% Completed
Completed in 5.467391014099121 seconds
6.07% Completed
Completed in 5.249973297119141 seconds
6.08% Completed
6.09% Completed
Completed in 6.34006142616272 seconds
6.11% Completed
Completed in 6.315118789672852 seconds
6.12% Completed
Completed in 3.4707281589508057 seconds
6.13% Completed
Completed in 6.1824564933776855 seconds
6.14% Completed
Completed in 3.875606060028076 seconds
6.15% Completed
6.16% Completed
Completed in 6.453735113143921 seconds
6.18% Completed
Completed in 6.454786062240601 seconds
6.19% Completed
6.20% Completed
Completed in 6.5345139503479 seconds
6.21% Completed
Completed in 6.364013910293579 seconds
6.22% Completed
Completed in 6.446774244308472 seconds

8.03% Completed
Completed in 6.396909713745117 seconds
8.05% Completed
8.06% Completed
Completed in 6.483643293380737 seconds
8.07% Completed
Completed in 5.384587049484253 seconds
8.08% Completed
Completed in 6.327069520950317 seconds
8.09% Completed
Completed in 5.735673189163208 seconds
8.10% Completed
Completed in 3.688145160675049 seconds
8.11% Completed
Completed in 4.959739685058594 seconds
8.13% Completed
Completed in 6.931466102600098 seconds
8.14% Completed
Completed in 6.337101936340332 seconds
8.15% Completed
Completed in 6.373003959655762 seconds
8.16% Completed
Completed in 4.416229248046875 seconds
8.17% Completed
8.18% Completed
8.20% Completed
Completed in 6.497638940811157 seconds
8.21% Completed
Completed in 5.804515600204468 seconds
8.22% Completed
Completed in 6.787886142730713 seconds
8.23% Completed
Completed in 6.289227485656738 seconds
8.24% Completed
Completed in 5.438466548919678 seconds
8.25% Completed
Completed in 6.231348752975464 seconds
8.27% Completed
C

...

92.70% Completed
Completed in 5.368658542633057 seconds
92.72% Completed
Completed in 3.630300283432007 seconds
92.73% Completed
Completed in 5.111372470855713 seconds
92.74% Completed
Completed in 5.331782102584839 seconds
92.75% Completed
Completed in 5.447446584701538 seconds
92.76% Completed
Completed in 5.3627002239227295 seconds
92.77% Completed
Completed in 5.133284330368042 seconds
92.79% Completed
Completed in 4.848033428192139 seconds
92.80% Completed
Completed in 3.646252393722534 seconds
92.81% Completed
Completed in 4.273587465286255 seconds
92.82% Completed
Completed in 4.367332458496094 seconds
92.83% Completed
Completed in 4.130963563919067 seconds
92.84% Completed
Completed in 4.423147916793823 seconds
92.85% Completed
Completed in 3.6303043365478516 seconds
92.87% Completed
Completed in 4.378326654434204 seconds
92.88% Completed
Completed in 4.234710693359375 seconds
92.89% Completed
Completed in 4.096091985702515 seconds
92.90% Completed
Completed in 4.18185138702392

Completed in 4.562781095504761 seconds
94.45% Completed
Completed in 3.610383987426758 seconds
94.46% Completed
Completed in 4.6725194454193115 seconds
94.47% Completed
Completed in 4.795188903808594 seconds
94.48% Completed
Completed in 4.74931001663208 seconds
94.49% Completed
Completed in 4.668552875518799 seconds
94.51% Completed
Completed in 4.754296779632568 seconds
94.52% Completed
Completed in 4.798180818557739 seconds
94.53% Completed
Completed in 4.546824216842651 seconds
94.54% Completed
Completed in 3.9933583736419678 seconds
94.55% Completed
Completed in 4.665537357330322 seconds
94.56% Completed
Completed in 4.0033323764801025 seconds
94.57% Completed
Completed in 4.694454193115234 seconds
94.59% Completed
Completed in 4.098024368286133 seconds
94.60% Completed
Completed in 4.128964900970459 seconds
94.61% Completed
Completed in 5.066436767578125 seconds
94.62% Completed
Completed in 5.035546779632568 seconds
94.63% Completed
Completed in 4.110983371734619 seconds
94.64% 

96.13% Completed
Completed in 4.337391376495361 seconds
96.14% Completed
Completed in 5.41453218460083 seconds
96.16% Completed
Completed in 4.295533180236816 seconds
96.17% Completed
Completed in 5.1762049198150635 seconds
96.18% Completed
Completed in 3.6333229541778564 seconds
96.19% Completed
Completed in 5.0256006717681885 seconds
96.20% Completed
Completed in 4.421215772628784 seconds
96.21% Completed
Completed in 3.9544434547424316 seconds
96.23% Completed
96.24% Completed
Completed in 5.043550252914429 seconds
96.25% Completed
Completed in 5.114306688308716 seconds
96.26% Completed
Completed in 5.217034101486206 seconds
96.27% Completed
Completed in 3.527582883834839 seconds
96.28% Completed
Completed in 4.930824041366577 seconds
96.29% Completed
Completed in 3.7968220710754395 seconds
96.31% Completed
Completed in 4.9577555656433105 seconds
96.32% Completed
Completed in 4.641598224639893 seconds
96.33% Completed
Completed in 3.8995800018310547 seconds
96.34% Completed
Complete

97.84% Completed
Completed in 4.68650484085083 seconds
97.85% Completed
Completed in 5.2719128131866455 seconds
97.86% Completed
Completed in 6.203423976898193 seconds
97.88% Completed
Completed in 3.75596284866333 seconds
97.89% Completed
Completed in 5.2060911655426025 seconds
97.90% Completed
Completed in 5.157215595245361 seconds
97.91% Completed
Completed in 5.146252155303955 seconds
97.92% Completed
Completed in 3.622354745864868 seconds
97.93% Completed
Completed in 4.0202836990356445 seconds
97.95% Completed
Completed in 5.73168420791626 seconds
97.96% Completed
Completed in 5.641925573348999 seconds
97.97% Completed
Completed in 5.46739387512207 seconds
97.98% Completed
Completed in 4.068105220794678 seconds
97.99% Completed
Completed in 5.6678290367126465 seconds
98.00% Completed
Completed in 6.536535739898682 seconds
98.01% Completed
Completed in 5.616997241973877 seconds
98.03% Completed
Completed in 5.685832738876343 seconds
98.04% Completed
Completed in 5.55120062828064 s

Completed in 6.243286609649658 seconds
99.54% Completed
Completed in 5.232990980148315 seconds
99.55% Completed
Completed in 5.323774099349976 seconds
99.56% Completed
Completed in 5.23999810218811 seconds
99.57% Completed
Completed in 5.146250009536743 seconds
99.58% Completed
Completed in 4.154881238937378 seconds
99.60% Completed
Completed in 5.253995895385742 seconds
99.61% Completed
Completed in 5.280865907669067 seconds
99.62% Completed
Completed in 3.8546998500823975 seconds
99.63% Completed
Completed in 3.7689316272735596 seconds
99.64% Completed
Completed in 4.4002392292022705 seconds
99.65% Completed
Completed in 4.139910697937012 seconds
99.67% Completed
Completed in 4.580762147903442 seconds
99.68% Completed
Completed in 3.881598711013794 seconds
99.69% Completed
Completed in 4.635653495788574 seconds
99.70% Completed
Completed in 4.686510801315308 seconds
99.71% Completed
Completed in 4.691434621810913 seconds
99.72% Completed
Completed in 3.8397717475891113 seconds
99.73%